In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [2]:
# fine tuning = 미세 조정 (모델을)
# fine tuning 을 위해선 pre-trained model이 필요
# pre trained 모델이란, 거대한 데이터(이미지넷)를 미리 학습한 모델을 의미
base_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet')


94765736/94765736 [==============================] - 5s 0us/step


In [3]:
# Fine tuning 순서
# 1. base model 은 모델 파라메터를 Freeze, Fully connected Layer만 학습
# optimizer의 LR = Default

# 2 - 1 우리의 데이터가 pre-trained 모델이 학습한 데이터보다 많을때,
# 2 - 1 - 1 카테고리가 비슷하다 -> base model의 50%
# 2 - 1 - 2 카테고리가 다르다 -> base model 50~100%

# 2 - 2 우리의 데이터가 pre-trained 모델이 학습한 데이터보다 적을때,
# base model의 10~30% 만 업데이트
# optimizer의 LR = Default / 10 ~ Default / 100
for layer in base_model.layers:
    layer.trainable = False

In [4]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, None, None, 3)        0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, None, None, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [5]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation = 'relu')(x)
output_layer = Dense(1, activation = 'sigmoid')(x)
our_resnet_model = Model(inputs = base_model.input, outputs = output_layer)
our_resnet_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, None, None, 3)        0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, None, None, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [6]:
our_resnet_model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
# google drive mount 후
!mkdir dataset
!unzip /content/drive/MyDrive/age.zip -d dataset

Archive:  /content/drive/MyDrive/age.zip
   creating: dataset/fifties/
  inflating: dataset/fifties/fifties_actor-001.jpeg  
  inflating: dataset/fifties/fifties_actor-002.jpeg  
  inflating: dataset/fifties/fifties_actor-003.jpeg  
  inflating: dataset/fifties/fifties_actor-004.jpeg  
  inflating: dataset/fifties/fifties_actor-005.jpeg  
  inflating: dataset/fifties/fifties_actor-006.jpeg  
  inflating: dataset/fifties/fifties_actor-007.jpeg  
  inflating: dataset/fifties/fifties_actor-008.jpeg  
  inflating: dataset/fifties/fifties_actor-009.jpeg  
  inflating: dataset/fifties/fifties_actor-010.jpeg  
  inflating: dataset/fifties/fifties_actor-011.jpeg  
  inflating: dataset/fifties/fifties_actor-012.jpeg  
  inflating: dataset/fifties/fifties_actor-013.jpeg  
  inflating: dataset/fifties/fifties_actor-014.jpeg  
  inflating: dataset/fifties/fifties_actor-015.jpeg  
  inflating: dataset/fifties/fifties_actor-016.jpeg  
  inflating: dataset/fifties/fifties_actor-017.jpeg  
  inflating

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    'dataset/',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 570 images belonging to 2 classes.


In [9]:
our_resnet_model.fit(train_generator, epochs=10)

Epoch 1/10
18/18 [==============================] - 12s 200ms/step - loss: 0.9858 - accuracy: 0.5263
Epoch 2/10
18/18 [==============================] - 2s 101ms/step - loss: 0.7313 - accuracy: 0.5842
Epoch 3/10
18/18 [==============================] - 2s 104ms/step - loss: 0.6728 - accuracy: 0.5860
Epoch 4/10
18/18 [==============================] - 2s 106ms/step - loss: 0.6663 - accuracy: 0.6123
Epoch 5/10
18/18 [==============================] - 2s 102ms/step - loss: 0.6539 - accuracy: 0.6228
Epoch 6/10
18/18 [==============================] - 2s 103ms/step - loss: 0.6627 - accuracy: 0.6088
Epoch 7/10
18/18 [==============================] - 2s 103ms/step - loss: 0.6306 - accuracy: 0.6386
Epoch 8/10
18/18 [==============================] - 2s 102ms/step - loss: 0.6474 - accuracy: 0.6228
Epoch 9/10
18/18 [==============================] - 2s 104ms/step - loss: 0.6324 - accuracy: 0.6421
Epoch 10/10
18/18 [==============================] - 2s 109ms/step - loss: 0.6246 - accuracy: 0.650

In [10]:
len(our_resnet_model.layers)

178

In [11]:
for layer in our_resnet_model.layers[:150]:
    layer.trainable = False
for layer in our_resnet_model.layers[150:]:
    layer.trainable = True

In [12]:
our_resnet_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, None, None, 3)        0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, None, None, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [13]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
our_resnet_model.compile(opt, loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
our_resnet_model.fit(train_generator, epochs=40)

Epoch 1/40
18/18 [==============================] - 11s 151ms/step - loss: 0.7447 - accuracy: 0.6298
Epoch 2/40
18/18 [==============================] - 2s 126ms/step - loss: 0.4411 - accuracy: 0.7877
Epoch 3/40
18/18 [==============================] - 2s 126ms/step - loss: 0.3210 - accuracy: 0.8614
Epoch 4/40
18/18 [==============================] - 2s 126ms/step - loss: 0.1558 - accuracy: 0.9351
Epoch 5/40
18/18 [==============================] - 2s 127ms/step - loss: 0.1479 - accuracy: 0.9404
Epoch 6/40
18/18 [==============================] - 2s 131ms/step - loss: 0.2024 - accuracy: 0.9228
Epoch 7/40
18/18 [==============================] - 2s 132ms/step - loss: 0.1502 - accuracy: 0.9333
Epoch 8/40
18/18 [==============================] - 2s 127ms/step - loss: 0.1477 - accuracy: 0.9404
Epoch 9/40
18/18 [==============================] - 2s 128ms/step - loss: 0.1982 - accuracy: 0.9175
Epoch 10/40
18/18 [==============================] - 2s 129ms/step - loss: 0.2040 - accuracy: 0.919